In [1]:
!pip install mxnet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 10.8 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1


In [2]:
import mxnet
mxnet.__version__

'1.9.1'

In [3]:
from mxnet.gluon import nn
from mxnet.gluon.data.vision import datasets, transforms
from tqdm import tqdm

import mxnet as mx
from mxnet import autograd, gluon, init

# Set the context to CPU or GPU
ctx = mx.gpu() if mx.context.num_gpus() > 0 else mx.cpu()

# Load and transform the CIFAR10 data
transform_train = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.2010]),
    ],
)
transform_test = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.2010]),
    ],
)
train_data = datasets.CIFAR10(train=True).transform_first(transform_train)
test_data = datasets.CIFAR10(train=False).transform_first(transform_test)

# Define the network architecture
net = nn.Sequential()
with net.name_scope():
    net.add(
        nn.Conv2D(channels=32, kernel_size=3, padding=1, activation="relu"),
        nn.MaxPool2D(pool_size=2, strides=2),
        nn.Conv2D(channels=16, kernel_size=3, padding=1, activation="relu"),
        nn.MaxPool2D(pool_size=2, strides=2),
        nn.Flatten(),
        nn.Dense(64, activation="relu"),
        nn.Dense(10),
    )
net.initialize(init=init.Xavier(), ctx=ctx)

# Define the loss function and optimizer
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()
trainer = gluon.Trainer(net.collect_params(), "adam", {"learning_rate": 0.0001})

# Train the network
batch_size = 128
epochs = 2
train_loader = gluon.data.DataLoader(train_data, batch_size, shuffle=True)
test_loader = gluon.data.DataLoader(test_data, batch_size)
for epoch in range(epochs):
    progress_bar = tqdm(train_loader, position=0, leave=True)
    train_loss, train_acc, n = 0.0, 0.0, 0
    for X, y in train_loader:
        X, y = X.as_in_context(ctx), y.as_in_context(ctx)
        with autograd.record():
            y_hat = net(X)
            loss = softmax_cross_entropy(y_hat, y)
        loss.backward()
        trainer.step(batch_size)
        current_loss = mx.nd.sum(loss).asscalar()
        train_loss += current_loss
        current_acc = mx.nd.sum(
            mxnet.ndarray.cast(y_hat.argmax(axis=1), dtype="int32") == y,
        ).asscalar()
        train_acc += current_acc
        n += y.size

        progress_bar.set_description(f"Epoch {epoch+1}/{epochs}")
        progress_bar.set_postfix(train_loss=round(current_loss, 3), train_acc=current_acc)
        progress_bar.update()

Epoch 2/2: 100%|█████████▉| 390/391 [00:41<00:00, 11.34it/s, train_acc=40, train_loss=111]

In [5]:
test_acc = 0.0

for X, y in test_loader:
    X, y = X.as_in_context(ctx), y.as_in_context(ctx)
    y_hat = net(X)
    current_acc = mx.nd.sum(
        mxnet.ndarray.cast(y_hat.argmax(axis=1), dtype="int32") == y,
    ).asscalar()
    test_acc += current_acc

test_acc / len(test_data)

0.4737